In [ ]:
import kplr 
client = kplr.API()
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pickle
import pprint

In [ ]:
kic = 6780873

In [ ]:
starsample = client.star(kic)
curves=starsample.get_light_curves(short_cadence=False, long_cadence=True, fetch=True, clobber=False)

In [ ]:
short_curves, long_curves = [], []

In [ ]:
for lc in curves:
    if "llc" in lc.filename:
        long_curves.append(lc)
    else:
        short_curves.append(lc)

In [ ]:
time, flux, flerr, quarter = np.array([]), np.array([]), np.array([]), np.array([])
for qq,lc in enumerate(long_curves): #same as KIC1718594 but changed short to long curves?
    print(lc.filename)
    with lc.open() as f:
        # read in and trim the data to good data
        thisdata = f[1].data
        thistime = thisdata["time"]
        thisflux = thisdata["pdcsap_flux"]
        good = np.isfinite(thistime)
        medianflux = np.nanmedian(thisflux)
        thisflux = thisflux[good] / medianflux
        thisflerr = thisdata["pdcsap_flux_err"][good] / medianflux
        thistime = thistime[good]
        thisquarter = np.zeros_like(thistime) + qq
        # patch bad fluxes
        bad = np.logical_not(np.isfinite(thisflux))
        thisflux[bad] = 1.
        thisflerr[bad] = 1000.
        # concatenate arrays
        time = np.concatenate((time, thistime))
        flux = np.concatenate((flux, thisflux))
        flerr = np.concatenate((flerr, thisflerr))
        quarter = np.concatenate((quarter, thisquarter))
print(time.shape, flux.shape, flerr.shape)
quarter = np.round(quarter).astype(int)

In [ ]:
plt.figure(figsize=(15,5))
plt.scatter(time,flux,marker=".",alpha=0.1, c=quarter) #not quarters like before but looks cool
plt.xlabel("time",fontsize=20)
plt.ylabel("flux",fontsize=20)
plt.title("INSERT TITLE")

In [ ]:
#need to work on grey band
plt.figure(figsize=(15,5))
plt.step(time,flux)
plt.xlabel("time",fontsize=20)
plt.ylabel("flux",fontsize=20)
plt.title("ZOOMED INSERT TITLE")
plt.xlim(130,170)

In [ ]:
dictionary1={"kic":kic,
             "time":time,
             "flux":flux,
             "flerr":flerr,
             "quarter":quarter}

In [ ]:
picklesave = open("rawdata.pickle","wb")
pickle.dump(dictionary1, picklesave)
picklesave.close()